In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from torch.nn.functional import pad, one_hot

from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

import Transformer
from Transformer import TransformerClassifier, MyDataset, create_padding_mask

Combine smaller files

In [ ]:
# ins = []
# for i in range (1,16):
#     name = "Train_Data_50_" +str(i)
#     with open(name,"rb") as file:
#         ins.append(pickle.load(file))

# out = pd.concat([x for x in ins])

# with open("Train_Data_m","wb") as file:
#         pickle.dump(out,file)

In [ ]:
# name = "Train_Data_m"
# with open(name,"rb") as file:
#     raw_data = pickle.load(file)

# raw_data.shape

Create DataLoader

In [ ]:
# Import Data
name = "Train_Data_m"
with open(name,"rb") as file:
    raw_data = pickle.load(file)


# Pad Data
data = raw_data["System_EWS"]
seq = [torch.from_numpy(run) for run in data]
#seq_trun = [torch.from_numpy(run[:,:6]).float() for run in data]

seq_padded = []
max_length = 650
for run in seq:
    pad_amount = max_length - run.shape[0]
    run_padded = pad(run, (0, 0, pad_amount, 0))
    seq_padded.append(run_padded)
input_data = torch.stack(seq_padded)

# Create one-hot labels
labels = [torch.tensor(label) for label in raw_data["null"]]
labels_t = torch.stack(labels)
labels_oh = one_hot(labels_t)


dataset = MyDataset(input_data, labels_oh)
batch_size = 50
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [2]:
input_data = torch.load("Test_Data")
labels_oh = torch.load("Test_Labels")
classes = torch.argmax(labels_oh,dim=1)
classes = classes.reshape(-1,1).float()

In [3]:
input_data.shape, labels_oh.shape, classes.shape

(torch.Size([62511, 600, 12]), torch.Size([62511, 2]), torch.Size([62511, 1]))

Import Test Data

In [4]:
dataset = MyDataset(input_data, classes)
batch_size = 100
dataloader = DataLoader(dataset, batch_size, shuffle=True)

Initialize Model and Select Parameters

In [5]:
dim_feedforward = 128
input_dim = 12  # Number of features
d_model = 64  # Transformer model dimension
num_heads = 8   # Number of attention heads
num_layers = 6  # Number of transformer layers
#num_classes = 2  # Number of classes for classification
dropout = 0.1   # Dropout rate

model = TransformerClassifier(input_dim, d_model, num_heads, num_layers, dim_feedforward, dropout)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [6]:
print(model)

TransformerClassifier(
  (projection): Linear(in_features=12, out_features=64, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (classification_head): Linear(in_features=64, out_features=1, bias=True)
)


Train Model

In [7]:
epochs = 5 # number of epochs

losses = []

for epoch in range(1,epochs+1):
    for batch_data, batch_labels in dataloader:
        # Forward pass with masking
        output = model(batch_data)
        # print(f"output shape: {output.shape},output: {output}")  # Output will have shape: [batch_size, num_classes]
        # print(f"labels: {batch_labels}")
        # Example of using CrossEntropyLoss for training
        loss = criterion(output, batch_labels)
        print(f"Loss: {loss.item()}")
        losses.append(loss.item())
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()

Loss: 1.2523925304412842
Loss: 0.7911041378974915
Loss: 0.44721633195877075
Loss: 0.5081697106361389
Loss: 0.4864215850830078
Loss: 0.4309339225292206
Loss: 0.4850313067436218
Loss: 0.4016149640083313
Loss: 0.39521339535713196
Loss: 0.2785756587982178
Loss: 0.2929535210132599
Loss: 0.3660261034965515
Loss: 0.27528154850006104
Loss: 0.34870320558547974
Loss: 0.3350238502025604
Loss: 0.2983570098876953
Loss: 0.4430692195892334
Loss: 0.45042702555656433
Loss: 0.3091234266757965
Loss: 0.4208530783653259
Loss: 0.2529304027557373
Loss: 0.30876094102859497
Loss: 0.20956988632678986
Loss: 0.35826876759529114
Loss: 0.24389547109603882
Loss: 0.29245585203170776
Loss: 0.1343485414981842
Loss: 0.1759498566389084
Loss: 0.2718880772590637
Loss: 0.24012543261051178
Loss: 0.17777790129184723
Loss: 0.10591600090265274
Loss: 0.15081185102462769
Loss: 0.11911630630493164
Loss: 0.1865147203207016
Loss: 0.20379865169525146
Loss: 0.09873127937316895
Loss: 0.2521936893463135
Loss: 0.17968252301216125
Loss: 0

: 

In [ ]:
# test_batch = input_data[:5000,:,:]
# test_labels = classes[:5000]
# raw_preds = model(batch_data)